### 초보자용

In [ ]:
#모듈 불러오기
import tensorflow as tf
import matplotlib.pyplot as plt

#데이터셋 불러오기
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#데이터셋 들여다 보기
print(len(x_train)) #데이터셋 길이 (60000)
print(type(x_train)) #데이터셋 유형 numpy.ndarray
print(x_train.shape) #데이터셋 모양 (60000, 28, 28)

In [ ]:
print(x_train)
print(y_train)

In [ ]:
print(x_test)
print(y_test)

In [ ]:
#이미지 그려보기
image = x_train[0]
image.shape
plt.imshow(image, "gray")

In [ ]:
#라벨과 이미지를 동시에 그려보기
plt.title(y_train[1])
plt.imshow(x_train[1], "gray")
plt.show()

In [ ]:
#데이터셋 전처리
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

#네트워크 구조 정의
model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="sigmoid"),
        tf.keras.layers.Dense(10, activation="softmax")]
)

#모델 컴파일
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy", metrics=["accuracy"])

#모델 훈련 및 평가
model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test,  y_test, verbose=2)

### 전문가용

In [ ]:
#모듈 불러오기
import tensorflow as tf

#데이터셋 전처리
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis] #CNN에서 필요함
x_test = x_test[..., tf.newaxis]		
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

#네트워크 구조 정의
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation="sigmoid")
        self.d2 = tf.keras.layers.Dense(10, activation="softmax")
    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [ ]:
#손실 함수, 최적화 함수 설정
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

#모델 평가지표 설정
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")
test_loss = tf.keras.metrics.Mean(name="test_loss")
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="test_accuracy")

In [ ]:
#학습 함수 정의
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

#테스트 함수 정의
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
#학습 및 테스트 실행
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)
    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)
    template = "에포크: {:.4f}, 손실: {:.4f}, 정확도: {:.4f}, 테스트 손실: {:.4f}, 테스트 정확도: {:.4f}"
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()